# API Notebook

In [1]:
#  import dependencies

import pandas as pd
import numpy as np
import json
import requests
from config import api_key


In [28]:
# requesting from api 
 
url = "https://api-nba-v1.p.rapidapi.com/gameDetails/"

headers = {
    'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
    'x-rapidapi-key': api_key
    }

response = requests.get(url, headers=headers)

response_json = response.json()

print(json.dumps(response_json, indent=4, sort_keys=True))

{
    "message": "Endpoint/gameDetails/ does not exist"
}


In [3]:
# creating json file to turn into df

# with open('test.json','w') as f:
#     json.dump(response_json,f)

I found the last game ID # that contained any data through guessing and process of elimination. Hypothetically, this is the max number of entries that could go into the table, but because the API is limited to 100 requests per day, I am choosing to keep the data frames smaller.

In [49]:

query_url = url + str(10861)
print(query_url)

q_response = requests.get(query_url,headers=headers).json()

print(json.dumps(q_response,indent=4,sort_keys=True))

https://api-nba-v1.p.rapidapi.com/gameDetails/10861
{
    "message": "You have exceeded the rate limit per minute for your plan, BASIC, by the API provider"
}


The cells below pull from the API to create dataframes from each of 'gameDetails', 'players/playerId', and 'teams/teamId'. The API-NBA from rapidapi.com is updated semi-regularly, so the code can be run over again to obtain the most recent data.

In [38]:
# game details info API to calls

details_base_url = "https://api-nba-v1.p.rapidapi.com/gameDetails/"

game_numbers = list(np.random.randint(10861, size=10))
game_numbers

game_info = []
game_not_found = []

for number in game_numbers:
    
    game_url = details_base_url + str(number)

    try:
        game_response = requests.get(game_url,headers=headers).json()

        gameId = game_response['api']['game'][0]['gameId']
        seasonYear = game_response['api']['game'][0]['seasonYear']
        arena = game_response['api']['game'][0]['arena']
        city = game_response['api']['game'][0]['city']
        country = game_response['api']['game'][0]['country']
        startTimeUTC = game_response['api']['game'][0]['startTimeUTC']
        gameDuration = game_response['api']['game'][0]['gameDuration']
        vTeam = game_response['api']['game'][0]['vTeam']['fullName']
        vTeamLeader = game_response['api']['game'][0]['vTeam']['leaders'][0]['playerId']
        awayScore = game_response['api']['game'][0]['vTeam']['score']['points']
        hTeam = game_response['api']['game'][0]['hTeam']['fullName']
        hTeamLeader = game_response['api']['game'][0]['hTeam']['leaders'][0]['playerId']
        homeScore = game_response['api']['game'][0]['hTeam']['score']['points']

        game_info.append({
            'Game ID':gameId,
            'Season Year':seasonYear,
            'Arena':arena,
            'City':city,
            'Country':country,
            'Start Time':startTimeUTC,
            'Game Duration':gameDuration,
            'Home Team':hTeam,
            'Home Score':homeScore,
            'Home Team Leader ID':hTeamLeader,
            'Away Team':vTeam,
            'Away Score':awayScore,
            'Away Team Leader ID':vTeamLeader
        })

# store any values not found and pass to keep the loop running 
    except:     
        game_not_found.append(gameId)
        pass


In [40]:
game_info_df = pd.DataFrame(game_info).set_index('Game ID')
game_info_df.head()

,Season Year,Arena,City,Country,Start Time,Game Duration,Home Team,Home Score,Home Team Leader ID,Away Team,Away Score,Away Team Leader ID
Game ID,,,,,,,,,,,,
7114,2019,Talking Stick Resort Arena,Phoenix,USA,2020-02-01T02:00:00.000Z,2:23,Phoenix Suns,107,459,Oklahoma City Thunder,111,4
711,2015,,,,2016-01-17T00:00:00.000Z,2:08,Charlotte Hornets,92,320,Milwaukee Bucks,105,361
601,2015,Air Canada Centre,Toronto,,2016-01-02T00:30:00.000Z,2:09,Toronto Raptors,104,136,Charlotte Hornets,94,538
4206,2017,Smoothie King Center,New Orleans,USA,2018-05-05T00:00:00.000Z,2:11,New Orleans Pelicans,119,126,Golden State Warriors,100,514
6776,2019,Pepsi Center,Denver,USA,2019-12-15T02:00:00.000Z,2:13,Denver Nuggets,110,38,Oklahoma City Thunder,102,4
2836,2016,,,,2017-05-26T00:30:00.000Z,2:21,Boston Celtics,102,68,Cleveland Cavaliers,135,265
1263,2015,,,,2016-04-04T00:00:00.000Z,2:14,Golden State Warriors,136,124,Portland Trail Blazers,111,319
8973,2020,Moda Center,Portland,USA,2021-04-03T02:00:00.000Z,2:11,Portland Trail Blazers,109,118,Milwaukee Bucks,127,20
1044,2015,,,,2016-03-06T22:00:00.000Z,2:39,Denver Nuggets,116,167,Dallas Mavericks,114,397


In [36]:
# players info API calls, 393 active players as of 2020

players_numbers = list(np.random.randint(393, size=10))

players_base_url = "https://api-nba-v1.p.rapidapi.com/players/playerId/"

player_info = []
player_not_found = []

for number in players_numbers:
    
    player_url = players_base_url + str(number)

    try:
        player_response = requests.get(player_url,headers=headers).json()
        
        playerId = player_response['api']['players'][0]['playerId']
        firstName = player_response['api']['players'][0]['firstName']
        lastName = player_response['api']['players'][0]['lastName']
        teamId = player_response['api']['players'][0]['teamId']
        country = player_response['api']['players'][0]['country']
        startNBA = player_response['api']['players'][0]['startNba']
        dateOfBirth = player_response['api']['players'][0]['dateOfBirth']
        height = player_response['api']['players'][0]['heightInMeters']
        weight = player_response['api']['players'][0]['weightInKilograms']

        player_info.append({
                'Player ID':playerId,
                'First Name':firstName,
                'Last Name':lastName,
                'Team ID':city,
                'Country':country,
                'Date of Birth':dateOfBirth,
                'Height':height,
                'Weight':weight
        })

# store any values not found and pass to keep the loop running 

    except:
        player_not_found.append(playerId)
        pass

        

In [37]:
players_df = pd.DataFrame(player_info).set_index('Player ID')
players_df.head()

,First Name,Last Name,Team ID,Country,Date of Birth,Height,Weight
Player ID,,,,,,,
116,Bryce,Cotton,Cleveland,,,,
222,Tobias,Harris,Cleveland,USA,1992-07-15,2.01,102.5
84,Alec,Burks,Cleveland,USA,1991-07-20,1.98,97.1
139,Boris,Diaw,Cleveland,France,1982-04-16,2.03,113.4
362,CJ,Miles,Cleveland,,,,


In [47]:
# teams calls 

teams_numbers = list(np.random.randint(30, size=10))

teams_base_url = "https://api-nba-v1.p.rapidapi.com/teams/teamId/"

team_info = []
team_not_found = []

for number in teams_numbers:
    
    team_url = teams_base_url + str(number)

    try:
        team_response = requests.get(team_url,headers=headers).json()
        
        teamId = team_response['api']['teams'][0]['teamId']
        fullName = team_response['api']['teams'][0]['fullName']
        nickName = team_response['api']['teams'][0]['nickName']
        shortName = team_response['api']['teams'][0]['shortName']
        city = team_response['api']['teams'][0]['city']

        team_info.append({
                'Team ID':teamId,
                'Team Name':fullName,
                'Nickname':nickName,
                'Abbreviation':shortName,
                'City':city
        })

# store any values not found and pass to keep the loop running 

    except:
        team_not_found.append(teamId)
        pass

        

In [ ]:
teams_df = pd.DataFrame(team_info).reset_index('Team ID')
teams_df.head()